# Power Plant Identification

In [ ]:
# Ensures that any methods/classes are reloaded if they are changed
%reload_ext autoreload
%autoreload 2

# Makes plots inline and higher resolution if using higher res monitors (uncomment second line otherwise)
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
# Load dependencies
from fastai.vision import ImageDataBunch, cnn_learner, ClassificationInterpretation, get_transforms, imagenet_stats, ShowGraph, models
from fastai.metrics import error_rate, accuracy
import matplotlib.pyplot as plt

In [ ]:
# Create a directory to store the data (only if it doesn't already exist)
!mkdir -p data/

## Power Plant Identification

In [ ]:
# Download the low resolution Landsat power plant imagery (only if it already hasn't been downloaded)
!wget -nc -O data/landsat_power_plants.zip https://github.com/kylebradbury/bc-tutorials/raw/master/data/landsat_power_plants.zip

In [ ]:
# Unzip the low resolution Landsat power plant imagery
!unzip -n data/landsat_power_plants.zip -d data/pp_landsat/

In [ ]:
path = 'data/pp_landsat/'

In [ ]:
batch_size = 5
image_size = 76 # pixels (for both height and width)

In [ ]:
data = ImageDataBunch.from_folder(path, ds_tfms=get_transforms(), size=image_size, bs=batch_size).normalize(imagenet_stats)

In [ ]:
data.show_batch(rows=3, figsize=(7,6))

In [ ]:
print(data.classes)

In [ ]:
learn = cnn_learner(data, models.resnet34, metrics=accuracy, callback_fns=ShowGraph, pretrained=True)

In [ ]:
#learn.unfreeze() # Uncomment this line to train ALL of the layers

In [ ]:
learn.fit_one_cycle(10, max_lr=slice(1e-4,1e-3))

In [ ]:
#learn.save('resnet34') # Use this line to save your trained model

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

In [ ]:
interp.plot_top_losses(16, figsize=(15,11))

In [ ]:
interp.plot_confusion_matrix(figsize=(6,6), dpi=120)

In [ ]:
interp.most_confused(min_val=2)